In [2]:
import math
import random
from tqdm import tqdm
import math
import numpy as np
from submodlib import FacilityLocationFunction, GraphCutFunction, DisparityMinFunction, DisparitySumFunction
import pickle
import time
import os
import timeit
from itertools import permutations
from torchvision import datasets, transforms

/raid/ganesh/namitha/miniconda3/envs/submodlib-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
dataset = "cifar10"

train_dataset = datasets.CIFAR10(root="../../data", train=True, download=True)
test_dataset = datasets.CIFAR10(root="../../data", train=False, download=True)


Files already downloaded and verified
Files already downloaded and verified


In [22]:
# import sys
# sys.path.append('..')
# from utils.feature_extrater.cifar_feature_extractor import extract_features_threaded_worker, extract_features_resnet_threaded_cifar

In [4]:
def generate_permutations(data):
    """Generates all permutations of a list."""
    for permutation in permutations(data):
        yield permutation

In [5]:
with open(f"../../../milo-base/cifar10/dataframe.pkl", "rb") as f:
    df = pickle.load(f)
    
groups = df.groupby('Label')
dataframes = [group for _, group in groups]

for i, df in enumerate(dataframes):
    df["Features"] = df["Features"].to_numpy()
    df["Index"] = df["Index"].to_numpy()

In [10]:
metric = "cosine"

In [27]:
func_list = ["facility-location", "disparity-min",  "disparity-sum", "graph-cut"]
subset_fraction_size = [0.5, 0.6, 0.5]

for order in generate_permutations(func_list):
    per_func_list = list(order)
    filename = ""
    for func in per_func_list:
        filename += func+"_"
    filename = filename[:-1]

    list_indexes = []
    for idx in range(len(func_list)-1):
        fraction_size = subset_fraction_size[idx]
        final_indexes = []
        for i, df in enumerate(dataframes):
            features = df["Features"]
            indexes = df["Index"]

            # can choose a different strategy 
            func = func_list[idx+i%2]

            if func=="facility-location":
                obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
            elif func=="disparity-min":
                obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="disparity-sum":
                obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="graph-cut":
                obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=0.45)
            else:
                raise Exception("Sorry, no submodlib function defined")
            
            S = obj.maximize(int(fraction_size*features.shape[0]), optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
            S = list(map(lambda tuple: tuple[0], S))

            features = features[S]
            indexes = indexes[S]

            final_indexes.extend(list(indexes))

            df["Features"] = features
            df["Index"] = indexes

        list_indexes.append(final_indexes)
        
    with open(f"{filename}.pkl", "wb") as f:
        pickle.dump(list_indexes, f)

In [8]:
print(type(S))

<class 'list'>
